In [0]:
# Imports
import json
from typing import Annotated, Any, Generator, Optional, Sequence, TypedDict, Union
from uuid import uuid4

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    convert_to_openai_messages,
)
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

In [0]:
supervisor_prompt = """

You are the Foodly Supervisor Agent.
You do not answer user questions directly.
Your job is to decide which specialist agent should handle the user’s latest request.

Available specialist agents:
- Order Agent: handles order lookups, statuses, details, cancellations.
- Policy Agent: handles questions about refunds, cancellations, loyalty, delivery, safety, privacy, promotions, or any other company policies.
- Escalation Agent: connects the user to a human when requested or when no other agent can resolve the issue.

Rules:
1. Read the user’s message carefully.
2. If the request clearly matches one of the above agents, delegate to that agent.
3. If no agent can solve the request, politely tell the user:
   "I’m sorry, I can’t handle that request. Would you like me to escalate this to a human support specialist?"
4. Never guess or make up answers yourself.
5. If multiple agents could handle it, pick the one that is the best fit (e.g., refund questions go to Policy Agent, not Order Agent).

Your output must be a routing decision: the name of the agent to call or a polite denial message.
"""

In [0]:
from langgraph_supervisor import create_supervisor

from orders_agent import AGENT as OrderAgent
from policy_agent import AGENT as PolicyAgent
from escalation_agent import AGENT as EscalationAgent
from databricks_langchain import (
    ChatDatabricks
)


LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)


# Ensure each agent has a unique name
OrderAgent.name = "OrderAgent"
PolicyAgent.name = "PolicyAgent"
EscalationAgent.name = "EscalationAgent"


supervisor = create_supervisor(
    model = llm ,
    agents=[OrderAgent, PolicyAgent, EscalationAgent],
    prompt=supervisor_prompt,
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [0]:
from helpers import LangGraphChatAgent
mlflow.langchain.autolog()
AGENT = LangGraphChatAgent(supervisor)
mlflow.models.set_model(AGENT)

In [0]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "where are my orders?"}]}
):
    print(event)